In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-sm"
dataPath = "/root/bc_data/Ddsm.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [5]:
metaData = bc.load_training_metadata(dataPath, verbose=True)

Before Balancing
----------------
benign 3164
malignant 1096

After Balancing
----------------
benign 3164
malignant 1096


In [6]:
# TODO: Actually load some representative data for model experimentation
maxData = 176
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 20, imgResize=imgResize)
print X_data.shape
print Y_data.shape

0.000000: A_0048_1.RIGHT_CC.LJPEG.pgm
0.113636: A_0128_1.RIGHT_MLO.LJPEG.pgm
0.113636: C_0176_1.RIGHT_MLO.LJPEG.pgm
0.113636: B_3647_1.RIGHT_MLO.LJPEG.pgm
0.113636: A_0146_1.LEFT_MLO.LJPEG.pgm
0.113636: B_3669_1.RIGHT_CC.LJPEG.pgm
0.113636: B_3062_1.RIGHT_MLO.LJPEG.pgm
0.113636: B_3645_1.RIGHT_MLO.LJPEG.pgm
0.113636: C_0229_1.RIGHT_CC.LJPEG.pgm
0.113636: B_3373_1.RIGHT_CC.LJPEG.pgm
0.113636: A_1140_1.LEFT_CC.LJPEG.pgm
0.113636: A_0510_1.LEFT_MLO.LJPEG.pgm
0.113636: C_0036_1.RIGHT_MLO.LJPEG.pgm
0.113636: A_0438_1.LEFT_CC.LJPEG.pgm
0.113636: B_3074_1.RIGHT_CC.LJPEG.pgm
0.113636: C_0072_1.RIGHT_MLO.LJPEG.pgm
0.113636: A_0469_1.RIGHT_MLO.LJPEG.pgm
0.113636: A_0566_1.LEFT_MLO.LJPEG.pgm
0.113636: A_1071_1.LEFT_MLO.LJPEG.pgm
0.113636: A_1172_1.LEFT_CC.LJPEG.pgm
0.113636: A_1154_1.RIGHT_MLO.LJPEG.pgm
0.113636: A_0526_1.RIGHT_MLO.LJPEG.pgm
0.113636: C_0102_1.LEFT_CC.LJPEG.pgm
0.113636: B_3378_1.RIGHT_CC.LJPEG.pgm
0.113636: B_3609_1.RIGHT_CC.LJPEG.pgm
0.113636: B_3515_1.RIGHT_MLO.LJPEG.pgm
0.113

## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [ ]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [ ]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

## Define and Load Trained Model

In [7]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 0, 'malignant': 1}
2


In [8]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,imgResize[0],imgResize[1]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [9]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [10]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

NameError: name 'X_test' is not defined

In [ ]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=200, epochs=30, verbosity=1)

In [ ]:
model.save_weights("dwdii-bc-150-v0-Cloud.hdf5", overwrite=True)